
# Metabolomics Preprocessing and Redox Ratios

Unpack Metabolomics Workbench study archives, harmonise metabolite identifiers, and compute redox ratio features (GSH/GSSG, NADPH/NADP+, MetSO/Met, lactate/pyruvate).


In [ ]:

# === Master Configuration ===
PROJECT_DIR = "/content/drive/MyDrive/NetsAnalysisProject"
RAW_METAB_DIR = f"{PROJECT_DIR}/data/raw/metabolomics"
EXTRACT_ROOT = f"{PROJECT_DIR}/data/processed/metabolomics/raw_tables"
OUTPUT_DIR = f"{PROJECT_DIR}/data/processed/metabolomics"
REDOX_TABLE = f"{OUTPUT_DIR}/redox_ratios.tsv"


In [ ]:

# Mount Drive & ensure directories
try:
    import google.colab  # type: ignore
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')
except ModuleNotFoundError:
    print('[info] Running outside Colab; ensure PROJECT_DIR is accessible.')

from pathlib import Path

for path in [Path(PROJECT_DIR), Path(RAW_METAB_DIR), Path(EXTRACT_ROOT), Path(OUTPUT_DIR)]:
    path.mkdir(parents=True, exist_ok=True)

print(f"[setup] Raw metabolomics archives under: {RAW_METAB_DIR}")


In [ ]:

# Install Python dependencies
!pip install --quiet pandas openpyxl pyarrow requests


In [ ]:

# Helper utilities for extraction and harmonisation
import zipfile
import tarfile
import pandas as pd
import numpy as np
import requests
from pathlib import Path
from typing import Dict, List, Tuple

STUDY_SYNONYMS = {
    'GSH': {'gsh', 'reduced glutathione', 'glutathione (reduced)', 'hmdb0000125'},
    'GSSG': {'gssg', 'oxidized glutathione', 'glutathione disulfide', 'hmdb0003336'},
    'NADPH': {'nadph', 'beta-nicotinamide adenine dinucleotide phosphate reduced', 'hmdb0000221'},
    'NADP': {'nadp', 'nadp+', 'hmdb0000219'},
    'MetSO': {'methionine sulfoxide', 'l-methionine sulfoxide', 'hmdb0003337'},
    'Met': {'methionine', 'l-methionine', 'hmdb0000696'},
    'Lactate': {'lactate', 'l-lactic acid', 'hmdb0000190'},
    'Pyruvate': {'pyruvate', 'pyruvic acid', 'hmdb0000243'},
}

RATIOS = {
    'GSH_GSSG': ('GSH', 'GSSG'),
    'NADPH_NADP': ('NADPH', 'NADP'),
    'MetSO_Met': ('MetSO', 'Met'),
    'Lactate_Pyruvate': ('Lactate', 'Pyruvate'),
}


def safe_extract(archive: Path, dest: Path):
    dest.mkdir(parents=True, exist_ok=True)
    if archive.suffix.lower() == '.zip':
        with zipfile.ZipFile(archive, 'r') as zf:
            zf.extractall(dest)
    elif archive.suffix.lower() in {'.tar', '.gz', '.bz2', '.tgz', '.tar.gz'}:
        with tarfile.open(archive, 'r:*') as tf:
            tf.extractall(dest)
    else:
        raise ValueError(f'Unsupported archive format: {archive}')


def find_tables(root: Path) -> List[Path]:
    patterns = ['*.csv', '*.tsv', '*.txt', '*.xlsx']
    files = []
    for pattern in patterns:
        files.extend(root.rglob(pattern))
    return files


def load_table(path: Path) -> pd.DataFrame:
    try:
        if path.suffix.lower() == '.csv':
            return pd.read_csv(path)
        if path.suffix.lower() == '.tsv' or path.suffix.lower() == '.txt':
            return pd.read_csv(path, sep='	')
        if path.suffix.lower() == '.xlsx':
            return pd.read_excel(path)
    except Exception as exc:
        print(f"[warn] Failed to load {path}: {exc}")
    return pd.DataFrame()


def normalise_column_names(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [str(c).strip() for c in df.columns]
    return df


def detect_hmdb_column(df: pd.DataFrame) -> str | None:
    for col in df.columns:
        if 'hmdb' in col.lower():
            return col
    return None


def detect_metabolite_column(df: pd.DataFrame) -> str | None:
    for col in df.columns:
        lower = col.lower()
        if 'metabolite' in lower or 'compound' in lower or 'analyte' in lower or 'name' == lower:
            return col
    return None


def melt_table(df: pd.DataFrame) -> pd.DataFrame:
    df = normalise_column_names(df)
    hmdb_col = detect_hmdb_column(df)
    name_col = detect_metabolite_column(df)
    id_col = hmdb_col if hmdb_col else name_col
    if id_col is None:
        raise ValueError('No HMDB or metabolite name column detected')
    meta_cols = [id_col]
    data_cols = [c for c in df.columns if c not in meta_cols]
    tidy = df.melt(id_vars=id_col, value_vars=data_cols, var_name='sample', value_name='intensity')
    tidy = tidy.dropna(subset=['intensity'])
    tidy['feature_id'] = tidy[id_col].astype(str).str.strip()
    tidy['sample'] = tidy['sample'].astype(str).str.strip()
    return tidy[['feature_id', 'sample', 'intensity']]


def match_feature(feature: str, target: str) -> bool:
    feature_clean = feature.lower()
    synonyms = STUDY_SYNONYMS[target]
    return feature_clean in synonyms


def harmonise_feature_id(feature: str) -> str:
    feat_lower = feature.lower()
    for label, synonyms in STUDY_SYNONYMS.items():
        if feat_lower in synonyms:
            return label
    return feature


def calculate_ratios(tidy: pd.DataFrame) -> pd.DataFrame:
    tidy = tidy.copy()
    tidy['harmonised_id'] = tidy['feature_id'].apply(harmonise_feature_id)
    ratios = []
    for ratio_name, (num_label, den_label) in RATIOS.items():
        numerator = tidy[tidy['harmonised_id'].str.lower() == num_label.lower()]
        denominator = tidy[tidy['harmonised_id'].str.lower() == den_label.lower()]
        if numerator.empty or denominator.empty:
            continue
        merged = numerator.merge(denominator, on='sample', suffixes=('_num', '_den'))
        merged['value'] = merged['intensity_num'] / merged['intensity_den']
        merged = merged[['sample', 'value']]
        merged['metric'] = ratio_name
        ratios.append(merged)
    if not ratios:
        return pd.DataFrame()
    combined = pd.concat(ratios, ignore_index=True)
    pivot = combined.pivot(index='sample', columns='metric', values='value')
    pivot.index.name = 'sample_id'
    return pivot


def process_archive(archive: Path) -> pd.DataFrame:
    dataset_name = archive.stem
    extract_dir = Path(EXTRACT_ROOT) / dataset_name
    safe_extract(archive, extract_dir)
    tables = find_tables(extract_dir)
    ratio_frames = []
    for table_path in tables:
        df = load_table(table_path)
        if df.empty:
            continue
        try:
            tidy = melt_table(df)
        except ValueError:
            continue
        ratio_df = calculate_ratios(tidy)
        if not ratio_df.empty:
            ratio_df['dataset'] = dataset_name
            ratio_frames.append(ratio_df)
    if not ratio_frames:
        print(f"[warn] No ratios generated for {archive}")
        return pd.DataFrame()
    merged = pd.concat(ratio_frames)
    merged = merged.groupby(level=0).mean()
    return merged


### Process all metabolomics archives

In [ ]:

from pathlib import Path

archives = sorted(Path(RAW_METAB_DIR).glob('*.zip'))
all_ratios = []

for archive in archives:
    print(f"==== Processing {archive.name} ====")
    ratios = process_archive(archive)
    if not ratios.empty:
        all_ratios.append(ratios)

if not all_ratios:
    raise RuntimeError('No ratio tables were generated. Check archive contents and column naming.')

combined = pd.concat(all_ratios, axis=0)
combined = combined.loc[:, sorted([c for c in combined.columns if c in RATIOS.keys()])]
combined.to_csv(REDOX_TABLE, sep='	')
combined


In [ ]:

print(f'Redox ratio table written to: {REDOX_TABLE}')
